# HW5: LSTM

In [45]:
import numpy as np
import torch
import torch.nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Pre-process the Training Data

training_firsts (and seconds) will be a lists of length 6036 each. Each element of these lists is a sentence vector, where each element corresponds to the words in the sentence. both `<s>` and `</s>` are included in each sentence vector. 

In [46]:
# TRAINING
training_size = 6036
training_firsts = []
training_secs = []
with open("bobsue-data/bobsue.seq2seq.train.tsv") as trainfile:
    for line in trainfile:
        separated = line.split()
        for i in range(len(separated)):
            if separated[i] == "</s>":
                sent1 = separated[:i + 1]
                sent2 = separated[i + 1:]
                break
        training_firsts.append(sent1)
        training_secs.append(sent2)


Now, we repeat for validation and testing sets:

In [47]:
# VALIDATION
validation_size = 750
valid_firsts = []
valid_secs = []
with open("bobsue-data/bobsue.seq2seq.dev.tsv") as valfile:
    for line in valfile:
        separated = line.split()
        for i in range(len(separated)):
            if separated[i] == "</s>":
                sent1 = separated[:i + 1]
                sent2 = separated[i + 1:]
                break
        valid_firsts.append(sent1)
        valid_secs.append(sent2)

# TESTING
testing_size = 750
test_firsts = []
test_secs = []
with open("bobsue-data/bobsue.seq2seq.test.tsv") as testfile:
    for line in testfile:
        separated = line.split()
        for i in range(len(separated)):
            if separated[i] == "</s>":
                sent1 = separated[:i + 1]
                sent2 = separated[i + 1:]
                break
        test_firsts.append(sent1)
        test_secs.append(sent2)

Import vocabulary and assign each vocab to a 200-d embedding:

In [78]:
#Pre-process Vocabulary
voc = []
with open('bobsue-data/bobsue.voc.txt') as vocfile:
    for line in vocfile:
        line = line.split()
        voc.append(line[0])


In [73]:
# GLOVE
word2idx = {}
idx = 0
vectors = []
with open('embedding-data/glove.6B.200d.txt') as embedfile:
    for embedline in embedfile:
        embedvec_str = embedline.split()
        embedword = embedvec_str[0]
        embedvec = np.array(embedvec_str[1:]).astype(np.double)
        word2idx[embedword] = idx
        idx += 1
        vectors.append(embedvec)

Now, let's use GloVe:

In [91]:
counter = 0
for vocword in voc:
    if vocword not in word2idx:
        #If a word from our vocab is not in GloVe, then we add an entry w/ random values
        #there are only 85 words in our vocab not in GloVe out of ~1500
        word2idx[vocword] = idx
        idx += 1
        rand_vec = np.random.rand(10) * 2 - 1
        vectors.append(rand_vec)
